In [2]:
import pandas as pd
import nltk

In [3]:
# Enron Emails can be found here https://www.cs.cmu.edu/~./enron/
text = pd.read_csv("data/emails.csv.zip", chunksize=10000)
df = next(text)

In [15]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [16]:
df.tail()

,file,message
9995,bass-e/_sent_mail/1047.,Message-ID: <21376194.1075854731510.JavaMail.e...
9996,bass-e/_sent_mail/1048.,Message-ID: <2401643.1075854731531.JavaMail.ev...
9997,bass-e/_sent_mail/1049.,Message-ID: <32002412.1075854731553.JavaMail.e...
9998,bass-e/_sent_mail/105.,Message-ID: <21429983.1075854709202.JavaMail.e...
9999,bass-e/_sent_mail/1050.,Message-ID: <20553983.1075854731575.JavaMail.e...


In [19]:
df = df.drop(columns=["file"])

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
message    10000 non-null object
dtypes: object(1)
memory usage: 78.2+ KB


In [123]:
message_structure_keys = [('Message-ID', False), ('Date', True), ('From', True), ('To', True), ('Subject', True), 
                        ('Mime-Version', False), ('Content-Type',False),('Content-Transfer-Encoding',False),
                        ('X-From',False),('X-To',False), ('X-cc',False), ('X-bcc',False),
                        ('X-Folder',False), ('X-Origin',False), ('X-FileName',False)]

In [126]:
def extract_value(key, line):
    return line.split(key + ':')[1].strip()

In [133]:
def transform_message(keys, message):
    message_structure = {}
    body = ''
    for key in keys:
        for line in message:
            if key[0] in line:
                if key[1]:
                    message_structure.update({key[0]:extract_value(key[0], line)})
                message.remove(line)
                break
            elif line:
                body = body + ' ' + line
    message_structure.update({'body':"".join(first_message).strip()})
    return message_structure

In [134]:
first_message = df.message[0].split('\n')

In [135]:
transform_message(message_structure_keys, first_message)

{'Date': 'Mon, 14 May 2001 16:39:00 -0700 (PDT)',
 'From': 'phillip.allen@enron.com',
 'To': 'tim.belden@enron.com',
 'Subject': '',
 'body': 'Here is our forecast'}

In [131]:
message_structure

{'Date': 'Mon, 14 May 2001 16:39:00 -0700 (PDT)',
 'From': 'phillip.allen@enron.com',
 'To': 'tim.belden@enron.com',
 'Subject': '',
 'body': 'Here is our forecast'}